In [1]:
import os
import pandas as pd
import json

In [2]:
# 폴더에 있는 모든 파일을 부른다
path_dir = "./"
file_list = [path_dir + file for file in os.listdir(path_dir) if file.endswith(".csv")]
file_list

['./1.csv',
 './2.csv',
 './output (3).csv',
 './output (4).csv',
 './output (5).csv']

In [3]:
# json list 형태 저장
json_list = []
for file in file_list:
    with open(file, 'r') as f:
        json_list.append(json.load(f))
print(len(json_list))

5


In [4]:
# 정답 key list 저장
key_list = list(json_list[0].keys())
len(key_list)

600

In [5]:
# data를 합친다
# best가 data0에 들어가있음(현재)
df_list = []
column_name_list = []
for i, json in enumerate(json_list):
    df = pd.DataFrame([json])
    df = df.transpose()
    column_name = 'data'+str(i)
    df.rename(columns={0:column_name},inplace=True)
    df_list.append(df)
    column_name_list.append(column_name)
    
merge_df = pd.concat(df_list,axis=1)

In [6]:
merge_df

,data0,data1,data2,data3,data4
mrc-1-000653,40억년전 지구,지구,40억년전 지구,사락사라,별하늘계의 변경에 위치한 외계행성 사만(サマーン)
mrc-1-001113,냉전,냉전 종식,냉전,냉전 종식,냉전
mrc-0-002191,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스
mrc-0-003951,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드
mrc-1-001272,프랑스,프랑스,프랑스,프랑스,프랑스
...,...,...,...,...,...
mrc-0-002989,재단 X,재단 X,재단 X,재단 X,재단 X
mrc-0-001804,자유당계 인사들,자유당계 인사들,자유당계 인사들,자유당계 인사들,자유당계 인사들
mrc-0-003411,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임
mrc-0-003436,디에프 항,주노 해변,디에프 항,주노 해변,디에프 항


In [21]:
column_name_list

['data0', 'data1', 'data2', 'data3', 'data4']

In [7]:
from collections import defaultdict

def count_list(data_list):
    # 개수를 key로 하여 data를 저장할 df
    count_df = defaultdict(list)
    
    # 중복 제거 list
    drop_duplicate_list = list(set(data_list))

    for element in drop_duplicate_list:
        data_cnt = data_list.count(element)
        # try:
        count_df[data_cnt].append(element)
        # except:
        #     count_df[data_cnt]=[element]
    
    return count_df

In [20]:
from datasets import load_from_disk, concatenate_datasets
from konlpy.tag import Mecab 
from tqdm.auto import tqdm
tokenizer = Mecab() 


def analysis_data():
    """
    train, validation answer의 형태소별 빈도수 저장
    """
    dataset = load_from_disk('../../data/train_dataset')
    train_dataset = dataset['train']
    val_dataset = dataset['validation']

    merged_dataset = concatenate_datasets([
        train_dataset.flatten_indices(),
        val_dataset.flatten_indices(),
    ])

    # print(merged_dataset.column_names)
    
    freq = defaultdict(int)
    for idx in tqdm(range(len(merged_dataset))):
        # print(merged_dataset['answers'][idx])

        results = tokenizer.pos(merged_dataset['answers'][idx]['text'][0])

        for result in results:
            freq[result[1]] += 1
    
    return freq
    
freq = analysis_data()

Loading cached processed dataset at ../../data/train_dataset/train/cache-fbc57aa6e699fb0c.arrow
Loading cached processed dataset at ../../data/train_dataset/validation/cache-d2fba0c42123b1d6.arrow


In [21]:
import pickle
with open('freq.pkl', 'wb') as handle:
    pickle.dump(freq, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('freq.pkl', 'rb') as handle:
    freq = pickle.load(handle)


In [22]:
J_list = ['JKS', 'JKS',	'JKC', 'JKC', 'JKG', 'JKG', 'JKO', 'JKO', 'JKB', 'JKB', 'JKV', 'JKV', 'JKQ', 'JKQ', 'JX', 'JC']

J_freq = []
for j in J_list: 
    J_freq.append(freq[j])

print(J_freq, sum(J_freq))

[75, 75, 0, 0, 297, 297, 82, 82, 120, 120, 1, 1, 2, 2, 46, 43] 1243


In [44]:
import numpy as np
def select_morpheme(answer_candidates):
    poses = [tokenizer.pos(answer) for answer in answer_candidates]
    # print(poses)
    answer = []
    
    for pos in poses:
        temp = []
        for p in pos:
            if p[-1] in J_list:
                temp.append(1)
            else:
                temp.append(0)
        answer.append(temp)
                
    # print(answer)
    # [[('시트로엥', 'NNP'), ('자동차', 'NNG'), ('회사', 'NNG')], [('시트로엥', 'NNP')]]
    # [[0, 0, 0], [0]]
    sum_first, sum_last = 0, 0

    for a in answer:
        sum_first += a[0]
        sum_last += a[-1]

    # 모든 답의 앞, 뒤에 조사가 포함된다면
    if sum_first == len(answer) or sum_last == len(answer):
        return answer_candidates[0]

    # 조사가 없는 것 먼저 반환
    for idx, a in enumerate(answer):
        if a[0] == 0 and a[-1] == 0:
            return a

In [45]:
"""
Hard Voting
"""

hard_voting = {}
# key = doc name
for key in key_list:
    # data 저장
    data_list = []
    for col in column_name_list:
        data_list.append(merge_df.loc[key][col])
    # data count
    count = count_list(data_list)

    answer_candidates = count[max(count.keys())]
    # print(answer_candidates)
    # 빈도수 동률 발생
    if len(answer_candidates) > 1:
        hard_voting[key] = select_morpheme(answer_candidates)
    elif len(answer_candidates) == 1:
        hard_voting[key] = answer_candidates[0]

In [46]:
hard_voting

{'mrc-1-000653': '40억년전 지구',
 'mrc-1-001113': '냉전',
 'mrc-0-002191': '대통령인 빌헬름 미클라스',
 'mrc-0-003951': '뉴질랜드',
 'mrc-1-001272': '프랑스',
 'mrc-1-000993': '다리',
 'mrc-0-005021': '대의제도',
 'mrc-1-000163': '하코네 산의 화산 폭발',
 'mrc-0-001283': '순조 11년(1811)',
 'mrc-0-004543': '고전도성 철',
 'mrc-0-000439': '리처드 말킨',
 'mrc-0-002895': '칼라치 전방 약250km 지점',
 'mrc-0-000535': '롭 포드',
 'mrc-1-001724': '노스햄프턴 교회',
 'mrc-0-000901': '언니',
 'mrc-0-001606': '석조 궁륭',
 'mrc-0-000266': '국민',
 'mrc-0-001326': '‘자유’지에',
 'mrc-0-000032': '국방부 보고서',
 'mrc-0-005215': '입석대',
 'mrc-0-005407': '숙의 정씨',
 'mrc-0-003683': '국가인권위원회',
 'mrc-0-003644': '이탈리아',
 'mrc-0-002835': '망자',
 'mrc-0-000049': '전통적인 서사',
 'mrc-1-001829': '화흡',
 'mrc-1-001662': '시라카와 성 공방전',
 'mrc-0-001206': '2004년',
 'mrc-0-004007': '신시내티대학교',
 'mrc-1-000418': '일등바위',
 'mrc-0-003133': '불법 어업',
 'mrc-0-004646': '건늠선',
 'mrc-0-001058': '카필라 성',
 'mrc-0-002361': '독립',
 'mrc-0-004830': '13개',
 'mrc-0-002762': '경복사',
 'mrc-0-000395': '후쿠도메 시게루 중장',
 'mrc-0-00166